1. Project Setup

In [ ]:
import os
# select a TensorFlow 2 detection model for use
# this is customizable - select a model here: https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md
CUSTOM_MODEL_NAME = 'my_ssd_resnet50' 
PRETRAINED_MODEL_NAME = 'ssd_resnet50_v1_fpn_640x640_coco17_tpu-8'
PRETRAINED_MODEL_URL = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz'
TF_RECORD_SCRIPT_NAME = 'generate_tfrecord.py'
LABEL_MAP_NAME = 'label_map.pbtxt'

# set references to paths & files
paths = {
	'assets_PATH': os.path.join('Tensorflow', 'assets'),   # assets directory, contains everything for the project
	'SCRIPTS_PATH': os.path.join('Tensorflow','scripts'),
	'APIMODEL_PATH': os.path.join('Tensorflow','models'),
	'ANNOTATION_PATH': os.path.join('Tensorflow', 'assets','annotations'),
	'IMAGE_PATH': os.path.join('Tensorflow', 'assets','images'),
	'MODEL_PATH': os.path.join('Tensorflow', 'assets','models'), 	# contains the model imported above
	'PRETRAINED_MODEL_PATH': os.path.join('Tensorflow', 'assets','pre-trained-models'),
	'CHECKPOINT_PATH': os.path.join('Tensorflow', 'assets','models',CUSTOM_MODEL_NAME), 
	'OUTPUT_PATH': os.path.join('Tensorflow', 'assets','models',CUSTOM_MODEL_NAME, 'export'), 
	'TFJS_PATH':os.path.join('Tensorflow', 'assets','models',CUSTOM_MODEL_NAME, 'tfjsexport'), 
	'TFLITE_PATH':os.path.join('Tensorflow', 'assets','models',CUSTOM_MODEL_NAME, 'tfliteexport'), 
	'PROTOC_PATH':os.path.join('Tensorflow','protoc')
}

files = {
	'PIPELINE_CONFIG':os.path.join('Tensorflow', 'assets','models', CUSTOM_MODEL_NAME, 'pipeline.config'),
	'TF_RECORD_SCRIPT': os.path.join(paths['SCRIPTS_PATH'], TF_RECORD_SCRIPT_NAME), 
	'LABELMAP': os.path.join(paths['ANNOTATION_PATH'], LABEL_MAP_NAME)
}

: 

In [42]:
for path in paths.values():
	if not os.path.exists(path):
		if os.name == 'posix': # linux
			%mkdir -p {path}
		if os.name == 'nt': # windows 
			%mkdir {path}

2. Download Model from TF2 Model Zoo & Install TFOD

In [43]:
!pip3 install matplotlib
!pip3 install wget
import wget

In [44]:
# install current version of tensorflow-models to APIMODEL_PATH
if not os.path.exists(os.path.join(paths['APIMODEL_PATH'], 'research','object_detection')):
  !git clone https://github.com/tensorflow/models {paths['APIMODEL_PATH']}


Cloning into 'Tensorflow/models'...
remote: Enumerating objects: 96640, done.
remote: Counting objects: 100% (6529/6529), done.
remote: Compressing objects: 100% (3523/3523), done.
error: RPC failed; curl 18 HTTP/2 stream 5 was reset80.00 KiB/s
error: 15947 bytes of body are still expected
fetch-pack: unexpected disconnect while reading sideband packet
fatal: early EOF
fatal: fetch-pack: invalid index-pack output


In [55]:
# install Protocal Buffer & TensorFlow Object Detection API
if os.name=='posix':  # linux
	!brew install protobuf-c
	!cd Tensorflow/models/research && object_detection/protos/*.proto --python_out=. && pip install . && export PYTHONPATH=$PYTHONPATH:`pwd`:`pwd`/slim && cp object_detection/packages/tf2/setup.py . && python -m pip install . 
if os.name=='nt': # windows
    wget.download("https://github.com/protocolbuffers/protobuf/releases/download/v3.15.6/protoc-3.15.6-win64.zip")
    %move protoc-3.15.6-win64.zip {paths['PROTOC_PATH']}
    %cd {paths['PROTOC_PATH']} && tar -xf protoc-3.15.6-win64.zip
    os.environ['PATH'] += os.pathsep + os.path.abspath(os.path.join(paths['PROTOC_PATH'], 'bin'))   
    %cd Tensorflow/models/research && protoc object_detection/protos/*.proto --python_out=. && copy object_detection\\packages\\tf2\\setup.py setup.py && python setup.py build && python setup.py install
    %cd Tensorflow/models/research/slim && pip install -e . 
   
# verify installation 
VERIFICATION_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'builders', 'model_builder_tf2_test.py')
!python {VERIFICATION_SCRIPT}

To reinstall 1.5.0_2, run:
  brew reinstall protobuf-c
object_detection/protos/anchor_generator.proto: line 1: syntax: command not found
object_detection/protos/anchor_generator.proto: line 3: package: command not found
object_detection/protos/anchor_generator.proto: line 5: import: command not found
object_detection/protos/anchor_generator.proto: line 6: import: command not found
object_detection/protos/anchor_generator.proto: line 7: import: command not found
object_detection/protos/anchor_generator.proto: line 8: import: command not found
object_detection/protos/anchor_generator.proto: line 10: //: is a directory
object_detection/protos/anchor_generator.proto: line 11: //: is a directory
object_detection/protos/anchor_generator.proto: line 12: message: command not found
object_detection/protos/anchor_generator.proto: line 13: oneof: command not found
object_detection/protos/anchor_generator.proto: line 14: GridAnchorGenerator: command not found
object_detection/protos/anchor_generat

In [5]:
# for Apple Silicon: install tensorflow-metal for GPU use
!pip3 install tensorflow-metal

ERROR: Could not find a version that satisfies the requirement tensorflow-metal (from versions: none)
ERROR: No matching distribution found for tensorflow-metal


In [10]:
import object_detection
# pre-trained models to be leveraged
if os.name =='posix': # linux
	!brew install wget
	!wget {PRETRAINED_MODEL_URL}
	!mv {PRETRAINED_MODEL_NAME+'.tar.gz'} {paths['PRETRAINED_MODEL_PATH']}
	!cd {paths['PRETRAINED_MODEL_PATH']} && tar -zxvf {PRETRAINED_MODEL_NAME+'.tar.gz'}
if os.name == 'nt': # windows
	wget.download(PRETRAINED_MODEL_URL)
	!move {PRETRAINED_MODEL_NAME+'.tar.gz'} {paths['PRETRAINED_MODEL_PATH']}
	!cd {paths['PRETRAINED_MODEL_PATH']} && tar -zxvf {PRETRAINED_MODEL_NAME+'.tar.gz'}
    
    

To reinstall 1.21.4, run:
  brew reinstall wget
--2024-03-10 12:15:09--  http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 2607:f8b0:400b:803::201b, 2607:f8b0:400b:80c::201b, 2607:f8b0:400b:804::201b, ...
Connecting to download.tensorflow.org (download.tensorflow.org)|2607:f8b0:400b:803::201b|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 244817203 (233M) [application/x-tar]
Saving to: ‘ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz’

ssd_resnet50_v1_fpn 100%[===================>] 233.48M  18.8MB/s    in 10s     

2024-03-10 12:15:20 (22.8 MB/s) - ‘ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz’ saved [244817203/244817203]

x ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/
x ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/checkpoint/
x ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/checkpoint/ckpt-0.data-00000-of-00001
x ssd_resnet50_v1_fpn_640x640_c

2. Create Label Map

In [56]:
labels = [{'name':'ThumbsUp', 'id':1}, {'name':'ThumbsDown', 'id':2}, {'name':'Up', 'id':3}, {'name':'Down', 'id':4},{'name':'Left', 'id':5}, {'name':'Right', 'id':6}]

with open(files['LABELMAP'], 'w') as f:
	for label in labels:
		f.write('item { \n')
		f.write('\tname:\'{}\'\n'.format(label['name']))
		f.write('\tid:{}\n'.format(label['id']))
		f.write('}\n')

3. Generate TF Records

In [57]:
# use a TFRecord generator script by developer nicknochnack
if not os.path.exists(files['TF_RECORD_SCRIPT']):
  !git clone https://github.com/nicknochnack/GenerateTFRecord {paths['SCRIPTS_PATH']}
!python {files['TF_RECORD_SCRIPT']} -x {os.path.join(paths['IMAGE_PATH'], 'train')} -l {files['LABELMAP']} -o {os.path.join(paths['ANNOTATION_PATH'], 'train.record')} 
!python {files['TF_RECORD_SCRIPT']} -x {os.path.join(paths['IMAGE_PATH'], 'test')} -l {files['LABELMAP']} -o {os.path.join(paths['ANNOTATION_PATH'], 'test.record')} 

Cloning into 'Tensorflow/scripts'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 3 (delta 0), reused 1 (delta 0), pack-reused 0
Receiving objects: 100% (3/3), done.
Traceback (most recent call last):
  File "/Users/bks/Documents/Side Projects/gesture-detector/Tensorflow/scripts/generate_tfrecord.py", line 29, in <module>
    from object_detection.utils import dataset_util, label_map_util
  File "/Users/bks/Documents/Side Projects/gesture-detector/gesture-detector/lib/python3.12/site-packages/object_detection/utils/label_map_util.py", line 21, in <module>
    from object_detection.protos import string_int_label_map_pb2
ImportError: cannot import name 'string_int_label_map_pb2' from 'object_detection.protos' (/Users/bks/Documents/Side Projects/gesture-detector/gesture-detector/lib/python3.12/site-packages/object_detection/protos/__init__.py)
Traceback (most recent call last):
  File "/Users

4. Copy Model Config Files to Training Folder

In [ ]:
if os.name =='posix': # linux
	!cp {os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'pipeline.config')} {os.path.join(paths['CHECKPOINT_PATH'])}
if os.name == 'nt': # windows
	!copy {os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'pipeline.config')} {os.path.join(paths['CHECKPOINT_PATH'])}

5. Update Config Files for Transfer Learning

In [7]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

config = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])
config
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], "r") as f:
  proto_str = f.read()  
  text_format.Merge(proto_str, pipeline_config)
pipeline_config.model.ssd.num_classes = len(labels)
pipeline_config.train_config.batch_size = 4
pipeline_config.train_config.fine_tune_checkpoint = os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'checkpoint', 'ckpt-0')
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path= files['LABELMAP']
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [os.path.join(paths['ANNOTATION_PATH'], 'train.record')]
pipeline_config.eval_input_reader[0].label_map_path = files['LABELMAP']
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [os.path.join(paths['ANNOTATION_PATH'], 'test.record')]
config_text = text_format.MessageToString(pipeline_config)
with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], "wb") as f:
  f.write(config_text)  

ImportError: cannot import name 'eval_pb2' from 'object_detection.protos' (/Users/bks/Documents/Side Projects/gesture-detector/gesture-detector/lib/python3.12/site-packages/object_detection/protos/__init__.py)

6. Train & Evaluate the model

In [ ]:
# train the model
TRAINING_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'model_main_tf2.py')
command = "python {} --model_dir={} --pipeline_config_path={} --num_train_steps=10000".format(TRAINING_SCRIPT, paths['CHECKPOINT_PATH'],files['PIPELINE_CONFIG'])
print(command)
!{command}

# evaluate the model
command = "python {} --model_dir={} --pipeline_config_path={} --checkpoint_dir={}".format(TRAINING_SCRIPT, paths['CHECKPOINT_PATH'],files['PIPELINE_CONFIG'], paths['CHECKPOINT_PATH'])
print(command)
!{command}

python Tensorflow/models/research/object_detection/model_main_tf2.py --model_dir=Tensorflow/assets/models/my_ssd_resnet50 --pipeline_config_path=Tensorflow/assets/models/my_ssd_resnet50/pipeline.config --num_train_steps=10000
/opt/homebrew/Cellar/python@3.12/3.12.2_1/Frameworks/Python.framework/Versions/3.12/Resources/Python.app/Contents/MacOS/Python: can't open file '/Users/bks/Documents/Side Projects/gesture-detector/Tensorflow/models/research/object_detection/model_main_tf2.py': [Errno 2] No such file or directory
python Tensorflow/models/research/object_detection/model_main_tf2.py --model_dir=Tensorflow/assets/models/my_ssd_resnet50 --pipeline_config_path=Tensorflow/assets/models/my_ssd_resnet50/pipeline.config --checkpoint_dir=Tensorflow/assets/models/my_ssd_resnet50
/opt/homebrew/Cellar/python@3.12/3.12.2_1/Frameworks/Python.framework/Versions/3.12/Resources/Python.app/Contents/MacOS/Python: can't open file '/Users/bks/Documents/Side Projects/gesture-detector/Tensorflow/models/re

7. Load Train Model From Checkpoint

In [1]:
import os
import tensorflow as tf
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder
from object_detection.utils import config_util

# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(paths['CHECKPOINT_PATH'], 'ckpt-1')).expect_partial()

@tf.function
def detect_fn(image):
	image, shapes = detection_model.preprocess(image)
	prediction_dict = detection_model.predict(image, shapes)
	detections = detection_model.postprocess(prediction_dict, shapes)
	return detections

ImportError: cannot import name 'string_int_label_map_pb2' from 'object_detection.protos' (/Users/bks/Documents/Side Projects/gesture-detector/gesture-detector/lib/python3.12/site-packages/object_detection/protos/__init__.py)

8. Detect from an Image

In [ ]:
import cv2 
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

# detect objects from an image
# edit the image filename below

category_index = label_map_util.create_category_index_from_labelmap(files['LABELMAP'])
IMAGE_PATH = os.path.join(paths['IMAGE_PATH'], 'test', 'IMAGEPATH.jpg')
img = cv2.imread(IMAGE_PATH)
image_np = np.array(img)

input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
detections = detect_fn(input_tensor)

num_detections = int(detections.pop('num_detections'))
detections = {key: value[0, :num_detections].numpy()
              for key, value in detections.items()}
detections['num_detections'] = num_detections

# detection_classes should be ints.
detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

label_id_offset = 1
image_np_with_detections = image_np.copy()

viz_utils.visualize_boxes_and_labels_on_image_array(
            image_np_with_detections,
            detections['detection_boxes'],
            detections['detection_classes']+label_id_offset,
            detections['detection_scores'],
            category_index,
            use_normalized_coordinates=True,
            max_boxes_to_draw=5,
            min_score_thresh=.8,
            agnostic_mode=False)

plt.imshow(cv2.cvtColor(image_np_with_detections, cv2.COLOR_BGR2RGB))
plt.show()

ModuleNotFoundError: No module named 'cv2'

9. Real-Time Detections from Webcam


In [8]:

cap = cv2.VideoCapture(0)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

while cap.isOpened(): 
	ret, frame = cap.read()
	image_np = np.array(frame)
    
	input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
	detections = detect_fn(input_tensor)
	
	num_detections = int(detections.pop('num_detections'))
	detections = {key: value[0, :num_detections].numpy()
								for key, value in detections.items()}
	detections['num_detections'] = num_detections

	# detection_classes should be ints.
	detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

	label_id_offset = 1
	image_np_with_detections = image_np.copy()

	viz_utils.visualize_boxes_and_labels_on_image_array(
		image_np_with_detections,
		detections['detection_boxes'],
		detections['detection_classes']+label_id_offset,
		detections['detection_scores'],
		category_index,
		use_normalized_coordinates=True,
		max_boxes_to_draw=5,
		min_score_thresh=.8,
		agnostic_mode=False)

	cv2.imshow('object detection',  cv2.resize(image_np_with_detections, (800, 600)))
	
	if cv2.waitKey(10) & 0xFF == ord('q'):
		cap.release()
		cv2.destroyAllWindows()
		break

NameError: name 'cv2' is not defined

10. Saving the Graph

In [ ]:
# save model for future use
FREEZE_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'exporter_main_v2.py ')
command = "python {} --input_type=image_tensor --pipeline_config_path={} --trained_checkpoint_dir={} --output_directory={}".format(FREEZE_SCRIPT ,files['PIPELINE_CONFIG'], paths['CHECKPOINT_PATH'], paths['OUTPUT_PATH'])
print(command)
!{command}

python Tensorflow/models/research/object_detection/exporter_main_v2.py  --input_type=image_tensor --pipeline_config_path=Tensorflow/assets/models/my_ssd_resnet50/pipeline.config --trained_checkpoint_dir=Tensorflow/assets/models/my_ssd_resnet50 --output_directory=Tensorflow/assets/models/my_ssd_resnet50/export


11. Convert to TensorFlowJS

In [ ]:
# for use in web applications
!pip3 install tensorflowjs
command = "tensorflowjs_converter --input_format=tf_saved_model --output_node_names='detection_boxes,detection_classes,detection_features,detection_multiclass_scores,detection_scores,num_detections,raw_detection_boxes,raw_detection_scores' --output_format=tfjs_graph_model --signature_name=serving_default {} {}".format(os.path.join(paths['OUTPUT_PATH'], 'saved_model'), paths['TFJS_PATH'])
print(command)
!{command}

tensorflowjs_converter --input_format=tf_saved_model --output_node_names='detection_boxes,detection_classes,detection_features,detection_multiclass_scores,detection_scores,num_detections,raw_detection_boxes,raw_detection_scores' --output_format=tfjs_graph_model --signature_name=serving_default Tensorflow/assets/models/my_ssd_resnet50/export/saved_model Tensorflow/assets/models/my_ssd_resnet50/tfjsexport
/Users/bks/Library/CloudStorage/OneDrive-UniversityofWaterloo/Side Projects/object-detector/obj-detector/lib/python3.12/site-packages/tensorflowjs/read_weights.py:28: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  np.uint8, np.uint16, np.object, np.bool]
Traceback (most recent call last):
  File "/Users/bks/Library/CloudStorage/OneDrive-UniversityofWaterloo/Side Projects/object-detector/obj-detector/bin/tensorflowjs_converter", line 7, in <module>
    from tensorflowjs.converters.converter import pip_main
  File "/Users/bks/Library/CloudSto